In [1]:
import numpy as np
import pandas as pd
import os
import csv
import re
from dotenv import load_dotenv
import json
from openai import OpenAI
from urllib.parse import urlparse
import hashlib
import requests
import ast

In [2]:
path = '/home/cptaswadu/RESCUE-n8n/insurance'
load_dotenv(dotenv_path=os.path.join(path, ".env"))
openai_api_key = os.getenv("OPEN_AI_API_KEY")
perplexity_api_key = os.getenv("PERPLEXITY_API_KEY")
gpt_client = OpenAI(api_key=openai_api_key)

In [3]:

BASE_RESULT_DIR = "/home/cptaswadu/RESCUE-n8n/insurance/results/policy_retrieval_fin"
LLM_FOLDER_ROOT = os.path.join(BASE_RESULT_DIR, "llm_searched")
MANUAL_FOLDER = "/home/cptaswadu/RESCUE-n8n/insurance/insurance_policy"
RETRIEVAL_SUMMARY_CSV = f"{BASE_RESULT_DIR}/retrieval_summary.csv"
MD5_COMPARISON_CSV = f"{BASE_RESULT_DIR}/md5_comparison.csv"
KNOWN_LINKS_CSV ="/home/cptaswadu/RESCUE-n8n/insurance/provider_policy.csv"

os.makedirs(BASE_RESULT_DIR, exist_ok=True)
os.makedirs(LLM_FOLDER_ROOT, exist_ok=True)
os.makedirs(os.path.join(BASE_RESULT_DIR, "retrieval"), exist_ok=True)
os.makedirs(os.path.join(BASE_RESULT_DIR, "md5"), exist_ok=True)

In [4]:
def policy_retrieval_prompt_baseline(provider_name):
    """
    Retrieves all official links to genetic testing coverage policies for a provider without keyword filtering.
    Returns strictly formatted JSON with 'pdf_links' and 'webpage_links'.
    """
    return (
        f"Find all official policy documents about genetic testing coverage for '{provider_name}'."
        f"Include every relevant policy, even if there are multiple documents."
        "Determine whether each link is a PDF or an HTML webpage based on its format or extension."
        "Group them accordingly in the response.\n\n"

        "Only include links from official sources such as the insurance company's website or regulatory bodies. "
        "Exclude links from news articles, blog posts, or discussion forums. "

        "If the policy is available as a PDF, return the direct PDF link under the key \"pdf_links\". "
        "If the policy is available only as an HTML webpage, return the webpage URL under the key \"webpage_links\". "
        "The response must be strictly in JSON format with two single keys: "
        "\"pdf_links\", containing an array of valid PDF URLs, and "
        "\"webpage_links\", containing an array of valid webpage URLs. "
        "Do not include any additional text or explanations—only the JSON object."
    )

def policy_retrieval_prompt_keyword_checked_document(provider_name):
    """
    Retrieves links only if the documents contain specific genetic-related keywords and excludes irrelevant content.
    """
    return (
        f"Find and list all official links to policy documents that describe genetic testing coverage for the insurance provider '{provider_name}'. "
        "Only include documents if they contain at least one of the following key terms: "
        "'coverage policy', 'medical policy', 'clinical policy', 'WES', 'WGS', 'BRCA', 'Duchenne muscular dystrophy', "
        "'hereditary cancer', 'genetic testing', 'lynch syndrome', or 'pharmacogenetic'. "
        "Include every relevant policy, even if there are multiple documents."
        "Exclude any documents that contain the phrase 'providal guideline', or that are press releases, claim forms, newsletters, blog posts, or provider manuals."
        "Only include links from official sources such as the insurance company’s website or regulatory bodies. "

        "Determine whether each link is a PDF or an HTML webpage based on its format or extension."
        "Group them accordingly in the response.\n\n"
        "If a document is available as a downloadable PDF, return the full direct PDF link under the key 'pdf_links'. "
        "If the document is only available as a webpage, return the full URL under the key 'webpage_links'. "
        "The JSON response must follow this exact format: "
        "{\"pdf_links\": [list of direct PDF links], \"webpage_links\": [list of webpage URLs]}. "
        "If no qualifying documents are found, return empty lists. "
        "Do not include any explanation, markdown, natural language, or formatting — only return the raw JSON object."
    )

def policy_retrieval_prompt_keyword_verified_links(provider_name):
    """
    Added stricter requirements for URL validity and official policy page confirmation.
    """
    return (
        f"Find and list all official links to policy documents that describe genetic testing coverage for the insurance provider '{provider_name}'. "
        "Only include documents if they contain at least one of the following key terms: "
        "'coverage policy', 'medical policy', 'clinical policy', 'WES', 'WGS', 'BRCA', 'Duchenne muscular dystrophy', "
        "'hereditary cancer', 'genetic testing', 'lynch syndrome', or 'pharmacogenetic'. "
        "Include every relevant policy, even if there are multiple documents."
        "Exclude any documents that contain the phrase 'providal guideline', or that are press releases, claim forms, newsletters, blog posts, or provider manuals."
        "Only include links from official sources such as the insurance company’s website or regulatory bodies, with official PDF links or official HTML policy pages."
        "Determine whether each link is a PDF or an HTML webpage based on its format or extension."
        "Group them accordingly in the response.\n\n"
        "If a document is available as a downloadable PDF, return the full direct PDF link under the key 'pdf_links'. "
        "If the document is only available as a webpage, return the full URL under the key 'webpage_links'. "
        "The JSON response must follow this exact format: "
        "{\"pdf_links\": [list of direct PDF links], \"webpage_links\": [list of webpage URLs]}. "
        "Make sure the lists contain only valid, existing URLs. If no documents are found, return empty lists. "
        "Do not include any explanation, markdown, natural language, or formatting — only return the raw JSON object."
    )

prompt_functions = {
    "baseline": policy_retrieval_prompt_baseline,
    "keyword": policy_retrieval_prompt_keyword_checked_document,
    "verified": policy_retrieval_prompt_keyword_verified_links
}


In [5]:
def download_pdf(url, save_path):
    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        with open(save_path, "wb") as f:
            f.write(response.content)
        print(f"✅ Downloaded PDF: {save_path}")
        return True
    except Exception as e:
        print(f"❌ Failed to download PDF from {url}: {e}")
        return False

def calculate_md5(file_path):
    """Calculate MD5 hash of a file"""
    hash_md5 = hashlib.md5()
    try:
        with open(file_path, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_md5.update(chunk)
        return hash_md5.hexdigest()
    except Exception as e:
        print(f"❌ Failed to calculate MD5 for {file_path}: {e}")
        return None

def load_existing_records(manual_pdf_folder, known_links_csv):
    """Load existing records from provider-specific PDF folders and CSV link list"""
    records = {
        "known_pdfs": {},  # {provider: {md5: file_path}}
        "known_webpages": {}  # {provider: [url1, url2, ...]}
    }
    
    # Load existing PDF files by provider
    if manual_pdf_folder and os.path.exists(manual_pdf_folder):
        print(f"📂 Loading known PDFs from: {manual_pdf_folder}")
        
        # 각 보험사 폴더별로 처리
        for provider_folder in os.listdir(manual_pdf_folder):
            provider_path = os.path.join(manual_pdf_folder, provider_folder)
            
            if os.path.isdir(provider_path):
                provider_name = provider_folder.replace("_", " ")
                records["known_pdfs"][provider_name] = {}
                
                for root, _, files in os.walk(provider_path):
                    for file in files:
                        if file.endswith('.pdf'):
                            file_path = os.path.join(root, file)
                            md5 = calculate_md5(file_path)
                            if md5:
                                records["known_pdfs"][provider_name][md5] = file_path
                
                print(f"   Found {len(records['known_pdfs'][provider_name])} PDFs for {provider_name}")
    
    # Load known webpage links from CSV
    if known_links_csv and os.path.exists(known_links_csv):
        print(f"📂 Loading known webpage links from: {known_links_csv}")
        try:
            df = pd.read_csv(known_links_csv)
            for _, row in df.iterrows():
                provider = row['Providers']  # 'provider' -> 'Providers'
                source = row['Source']  # Source 컬럼 추가
                links_str = row['Links']  # 'links' -> 'Links'
                
                # HTML 링크만 처리
                if source == 'html' and pd.notna(links_str) and str(links_str).strip():
                    links = [link.strip() for link in str(links_str).split('\n') if link.strip()]
                    records["known_webpages"][provider] = links
                else:
                    records["known_webpages"][provider] = []
        except Exception as e:
            print(f"❌ Failed to load webpage links CSV: {e}")
    
    return records



In [6]:
def query_llm_for_providers(messages, model="openai", openai_client=None, perplexity_api_key=None, max_retries=3):
    def call_openai():
        prompt = messages[-1]["content"]  # Extract user prompt for OpenAI web search
        response = openai_client.responses.create(
            model="gpt-4o",
            input=messages,
            tools=[{"type": "web_search_preview"}]
        )
        return response.output_text.strip()

    def call_perplexity():
        headers = {
            "Authorization": f"Bearer {perplexity_api_key}",
            "Content-Type": "application/json"
        }
        data = {
            "model": "sonar-pro",
            "messages": messages
        }
        url = "https://api.perplexity.ai/chat/completions"
        res = requests.post(url, headers=headers, json=data)
        if res.status_code == 200:
            return res.json()["choices"][0]["message"]["content"].strip()
        else:
            raise Exception(f"Perplexity error: {res.status_code} - {res.text}")

    for attempt in range(1, max_retries + 1):
        try:
            print(f"🔁 Attempt {attempt} ({model})...")
            return call_perplexity() if model == "perplexity" else call_openai()
        except Exception as e:
            print(f"❌ Attempt {attempt} failed: {e}")
    return None


In [7]:
def extract_provider_json(response_text):
    original = response_text.strip()

    # Step 1: Try direct JSON
    try:
        result = json.loads(original)
        if isinstance(result, dict) and "pdf_links" in result:
            return result
    except json.JSONDecodeError:
        pass

    # Step 2: Try cleanup of ```json blocks
    response_text = re.sub(r"^```json\s*|\s*```$", "", original, flags=re.IGNORECASE).strip()
    
    # Step 3: Try parsing again
    try:
        result = json.loads(response_text)
        if isinstance(result, dict) and "pdf_links" in result:
            return result
    except json.JSONDecodeError:
        pass

    # Step 4: Find first JSON block in messy response
    json_match = re.search(r"(\{[\s\S]*?\})", original)
    if json_match:
        try:
            result = json.loads(json_match.group(1))
            if isinstance(result, dict) and "pdf_links" in result:
                return result
        except json.JSONDecodeError as e:
            print("⚠️ Still invalid JSON block:", e)

    print("⚠️ Could not parse JSON. Using fallback empty provider list.")
    return {
        "pdf_links": [],
        "webpage_links": []
    }

In [8]:
def retrieve_and_save_policy(provider, prompt_fn, model="openai", prompt_name="baseline", iteration=1,
                           openai_client=None, perplexity_api_key=None,
                           manual_pdf_folder=None, known_links_csv=None,
                           base_output_dir="results"):
    '''
    Retrieves genetic testing policy links for the given provider and compares with known data.
    Returns a dictionary containing retrieval result summary with comparison metrics.
    '''
    print(f"\n🔍 [Iteration {iteration}] Searching for: {provider}")
    
    messages = [
        {"role": "system", "content": "You are a helpful and precise research assistant."},
        {"role": "user", "content": prompt_fn(provider)}
    ]

    try:
        # LLM 응답 받기
        response_text = query_llm_for_providers(
            messages, model=model, openai_client=openai_client, perplexity_api_key=perplexity_api_key
        )
        print(f"\n🧾 {model.upper()} raw response for '{provider}':\n{response_text}\n")

        download_folder = os.path.join(base_output_dir, f"iteration_{iteration}", "downloads", 
                                     model, prompt_name, provider.replace(" ", "_"))
        os.makedirs(download_folder, exist_ok=True)

        raw_response_folder = os.path.join(download_folder, "raw_responses")
        os.makedirs(raw_response_folder, exist_ok=True)
        with open(os.path.join(raw_response_folder, f"{provider}_raw_response.txt"), "w") as f:
            f.write(response_text)
        
        result_json = extract_provider_json(response_text)

        pdf_links = result_json.get("pdf_links", [])
        webpage_links = result_json.get("webpage_links", [])

        # 기존 데이터 로드 (비교용)
        known_records = load_existing_records(manual_pdf_folder, known_links_csv)
        
        provider_known_pdfs = known_records["known_pdfs"].get(provider, {})
        provider_known_webpages = known_records["known_webpages"].get(provider, [])

        if not provider_known_pdfs and not provider_known_webpages:
            print(f"⚠️  No known data for {provider}")
        elif not provider_known_pdfs:
            print(f"⚠️  No known PDFs for {provider}")
        elif not provider_known_webpages:
            print(f"⚠️  No known webpages for {provider}")
        
        # 해당 보험사의 known PDFs와 webpages 가져오기
        print(f"📋 Known data for {provider}:")
        print(f"   Known PDFs: {len(provider_known_pdfs)}")
        print(f"   Known webpages: {len(provider_known_webpages)}")
        
        # PDF 다운로드 및 MD5 비교
        downloaded_pdfs = 0
        pdf_matches = 0
        
        for link in pdf_links:
            filename = os.path.basename(link.split("?")[0])
            save_path = os.path.join(download_folder, filename)
            
            if download_pdf(link, save_path):
                downloaded_pdfs += 1
                
                # MD5 비교 (해당 보험사의 PDF와만 비교)
                file_md5 = calculate_md5(save_path)
                if file_md5 and file_md5 in provider_known_pdfs:
                    pdf_matches += 1
                    print(f"✅ PDF Match: {filename} (MD5: {file_md5[:8]}...)")
                    print(f"   Known file: {provider_known_pdfs[file_md5]}")
                else:
                    print(f"❌ PDF New/Unmatched: {filename}")
        
        # 웹페이지 링크 비교 (해당 보험사의 링크와만 비교)
        webpage_matches = len([link for link in webpage_links if link in provider_known_webpages])
        
        for link in webpage_links:
            if link in provider_known_webpages:
                print(f"✅ Webpage Match: {link}")
            else:
                print(f"❌ Webpage New/Unmatched: {link}")

        # return 문이 for 루프 밖에 있어야 함!
        return {
            "Provider": provider,
            "Iteration": iteration,
            "PDF Links": json.dumps(pdf_links),
            "PDF Count": len(pdf_links),
            "Downloaded PDFs": downloaded_pdfs,
            "PDF Matches": pdf_matches,
            "Known PDFs": len(provider_known_pdfs),
            "Webpage Links": json.dumps(webpage_links),
            "Webpage Count": len(webpage_links),
            "Webpage Matches": webpage_matches,
            "Known Webpages": len(provider_known_webpages),
            "Total Links": len(pdf_links) + len(webpage_links),
            "Total Matches": pdf_matches + webpage_matches,
            "Total Known": len(provider_known_pdfs) + len(provider_known_webpages)
        }

    except Exception as e:
        print(f"❌ Error processing {provider}: {e}")
        return {
            "Provider": provider,
            "Iteration": iteration,
            "PDF Links": "[]",
            "PDF Count": 0,
            "Downloaded PDFs": 0,
            "PDF Matches": 0,
            "Known PDFs": 0,
            "Webpage Links": "[]",
            "Webpage Count": 0,
            "Webpage Matches": 0,
            "Known Webpages": 0,
            "Total Links": 0,
            "Total Matches": 0,
            "Total Known": 0
        }


In [6]:
def summarize_policy_retrieval_with_metrics(providers, prompt_fn, model="openai", prompt_name="baseline", 
                                          iteration=1, openai_client=None, perplexity_api_key=None,
                                          manual_pdf_folder=None, known_links_csv=None,
                                          base_output_dir="results"):
    """
    Summarizes retrieval results with Recall/Precision metrics for one iteration.
    """
    print(f"\n📊 [Iteration {iteration}] Starting retrieval summary for {len(providers)} providers...")
    
    results = []
    
    # 각 보험사별로 데이터 수집
    for provider in providers:
        result = retrieve_and_save_policy(
            provider, prompt_fn, model=model, prompt_name=prompt_name, iteration=iteration,
            openai_client=openai_client, perplexity_api_key=perplexity_api_key,
            manual_pdf_folder=manual_pdf_folder, known_links_csv=known_links_csv,
            base_output_dir=base_output_dir
        )
        results.append(result)
    
    # DataFrame으로 변환
    df = pd.DataFrame(results)
    
    # 메트릭 계산 (보험사별 정확한 계산)
    total_pdf_found = df["PDF Count"].sum()
    total_pdf_matched = df["PDF Matches"].sum()
    total_pdf_known = df["Known PDFs"].sum()
    
    total_webpage_found = df["Webpage Count"].sum()
    total_webpage_matched = df["Webpage Matches"].sum()
    total_webpage_known = df["Known Webpages"].sum()
    
    # Overall metrics
    total_found = total_pdf_found + total_webpage_found
    total_matched = total_pdf_matched + total_webpage_matched
    total_known = total_pdf_known + total_webpage_known
    
    overall_recall = total_matched / total_known if total_known > 0 else 0
    overall_precision = total_matched / total_found if total_found > 0 else 0
    
    # Provider별 recall/precision 계산
    provider_recalls = []
    provider_precisions = []
    
    for _, row in df.iterrows():
        provider = row["Provider"]
        found = row["Total Links"]
        matched = row["Total Matches"]
        known = row["Total Known"]
        
        recall = matched / known if known > 0 else 0
        precision = matched / found if found > 0 else 0
        
        provider_recalls.append(recall)
        provider_precisions.append(precision)
        
        print(f"📈 {provider} Metrics:")
        print(f"   Found: {found}, Matched: {matched}, Known: {known}")
        print(f"   Recall: {recall:.4f}, Precision: {precision:.4f}")
    
    metrics = {
        "total_docs": df["Total Links"].sum(),
        "total_pdfs": df["PDF Count"].sum(),
        "total_webpages": df["Webpage Count"].sum(),
        "overall_recall": overall_recall,
        "overall_precision": overall_precision,
        "avg_recall_per_provider": round(sum(provider_recalls) / len(provider_recalls), 4) if provider_recalls else 0,
        "avg_precision_per_provider": round(sum(provider_precisions) / len(provider_precisions), 4) if provider_precisions else 0,
        "total_found": total_found,
        "total_matched": total_matched,
        "total_known": total_known
    }
    
    print(f"\n📊 Overall Metrics:")
    print(f"   Total Found: {total_found}")
    print(f"   Total Matched: {total_matched}")
    print(f"   Total Known: {total_known}")
    print(f"   Overall Recall: {overall_recall:.4f}")
    print(f"   Overall Precision: {overall_precision:.4f}")
    
    # 폴더 생성 및 저장
    results_folder = os.path.join(base_output_dir, f"iteration_{iteration}", model, prompt_name)
    
    # 1. 상세 결과 저장
    raw_results_folder = os.path.join(results_folder, "raw_results")
    os.makedirs(raw_results_folder, exist_ok=True)
    detailed_path = os.path.join(raw_results_folder, f"{model}_{prompt_name}_detailed.csv")
    df.to_csv(detailed_path, index=False)
    print(f"📄 Detailed results saved to: {detailed_path}")
    
    # 2. 메트릭 테이블 저장
    metrics_folder = os.path.join(results_folder, "metrics")
    os.makedirs(metrics_folder, exist_ok=True)
    metrics_path = os.path.join(metrics_folder, f"{model}_{prompt_name}_metrics.csv")
    
    # 최종 메트릭 테이블 생성
    final_metrics_df = pd.DataFrame([{
        "Metric": "Total documents (average #)",
        "Value": metrics["total_docs"]
    }, {
        "Metric": "Total PDFs (average #)",
        "Value": metrics["total_pdfs"]
    }, {
        "Metric": "Total web page (average #)",
        "Value": metrics["total_webpages"]
    }, {
        "Metric": "Overall Recall",
        "Value": metrics["overall_recall"]
    }, {
        "Metric": "Overall Precision",
        "Value": metrics["overall_precision"]
    }, {
        "Metric": "Average Recall per provider",
        "Value": metrics["avg_recall_per_provider"]
    }, {
        "Metric": "Average Precision per provider",
        "Value": metrics["avg_precision_per_provider"]
    }])
    
    final_metrics_df.to_csv(metrics_path, index=False)
    print(f"📄 Metrics table saved to: {metrics_path}")
    
    return {
        "detailed_results": df,
        "metrics": metrics,
        "iteration": iteration,
        "model": model,
        "prompt_name": prompt_name
    }


In [7]:
def compare_models_union_intersection(model_a, model_b, prompt_name, iteration=1, 
                                    base_output_dir="results"):
    """
    Compares MD5 union/intersection (PDFs) and URL union/intersection (webpages) between two models.
    """
    print(f"\n🔬 [Iteration {iteration}] Comparing {model_a} vs {model_b} ({prompt_name})...")
    
    def compute_md5_set_for_model(model_path):
        """Compute MD5 set for all PDFs in model path"""
        md5_set = set()
        
        if os.path.exists(model_path):
            for root, _, files in os.walk(model_path):
                for file in files:
                    if file.endswith('.pdf'):
                        file_path = os.path.join(root, file)
                        md5_hash = calculate_md5(file_path)
                        if md5_hash:
                            md5_set.add(md5_hash)
        
        return md5_set
    
    def get_webpage_links_from_results(model, prompt_name, iteration):
        """Extract all webpage links from detailed results CSV"""
        results_path = os.path.join(base_output_dir, f"iteration_{iteration}", model, prompt_name, 
                                  "raw_results", f"{model}_{prompt_name}_detailed.csv")
        
        webpage_links = set()
        if os.path.exists(results_path):
            try:
                df = pd.read_csv(results_path)
                for _, row in df.iterrows():
                    links_str = row.get("Webpage Links", "[]")
                    if links_str != "[]":
                        links = json.loads(links_str)
                        webpage_links.update(links)
            except Exception as e:
                print(f"❌ Failed to load webpage links from {results_path}: {e}")
        
        return webpage_links
    
    # PDF MD5 비교
    pdf_path_a = os.path.join(base_output_dir, f"iteration_{iteration}", "downloads", model_a, prompt_name)
    pdf_path_b = os.path.join(base_output_dir, f"iteration_{iteration}", "downloads", model_b, prompt_name)
    
    pdf_set_a = compute_md5_set_for_model(pdf_path_a)
    pdf_set_b = compute_md5_set_for_model(pdf_path_b)
    
    pdf_union = pdf_set_a | pdf_set_b
    pdf_intersection = pdf_set_a & pdf_set_b
    pdf_only_a = pdf_set_a - pdf_set_b
    pdf_only_b = pdf_set_b - pdf_set_a
    
    # 웹페이지 링크 비교
    webpage_set_a = get_webpage_links_from_results(model_a, prompt_name, iteration)
    webpage_set_b = get_webpage_links_from_results(model_b, prompt_name, iteration)
    
    webpage_union = webpage_set_a | webpage_set_b
    webpage_intersection = webpage_set_a & webpage_set_b
    webpage_only_a = webpage_set_a - webpage_set_b
    webpage_only_b = webpage_set_b - webpage_set_a
    
    print(f"📊 PDF Results:")
    print(f"   {model_a} unique PDFs: {len(pdf_set_a)}")
    print(f"   {model_b} unique PDFs: {len(pdf_set_b)}")
    print(f"   PDF Union: {len(pdf_union)}")
    print(f"   PDF Intersection: {len(pdf_intersection)}")
    
    print(f"📊 Webpage Results:")
    print(f"   {model_a} unique webpages: {len(webpage_set_a)}")
    print(f"   {model_b} unique webpages: {len(webpage_set_b)}")
    print(f"   Webpage Union: {len(webpage_union)}")
    print(f"   Webpage Intersection: {len(webpage_intersection)}")
    
    # 전체 통합 결과
    total_union = len(pdf_union) + len(webpage_union)
    total_intersection = len(pdf_intersection) + len(webpage_intersection)
    
    print(f"📊 Combined Results:")
    print(f"   Total Union: {total_union}")
    print(f"   Total Intersection: {total_intersection}")
    
    # 결과 저장
    results_folder = os.path.join(base_output_dir, f"iteration_{iteration}", model_a, prompt_name, "comparisons")
    os.makedirs(results_folder, exist_ok=True)
    
    comparison_path = os.path.join(results_folder, f"{model_a}_vs_{model_b}_{prompt_name}.csv")
    
    comparison_df = pd.DataFrame([{
        "Iteration": iteration,
        "Prompt": prompt_name,
        "Model A": model_a,
        "Model B": model_b,
        "Model A PDFs": len(pdf_set_a),
        "Model B PDFs": len(pdf_set_b),
        "PDF Union Count": len(pdf_union),
        "PDF Intersection Count": len(pdf_intersection),
        "PDF Only Model A": len(pdf_only_a),
        "PDF Only Model B": len(pdf_only_b),
        "Model A Webpages": len(webpage_set_a),
        "Model B Webpages": len(webpage_set_b),
        "Webpage Union Count": len(webpage_union),
        "Webpage Intersection Count": len(webpage_intersection),
        "Webpage Only Model A": len(webpage_only_a),
        "Webpage Only Model B": len(webpage_only_b),
        "Total Union Count": total_union,
        "Total Intersection Count": total_intersection,
        "PDF Overlap Percentage": round(len(pdf_intersection) / len(pdf_union) * 100, 2) if pdf_union else 0,
        "Webpage Overlap Percentage": round(len(webpage_intersection) / len(webpage_union) * 100, 2) if webpage_union else 0,
        "Overall Overlap Percentage": round(total_intersection / total_union * 100, 2) if total_union else 0
    }])
    
    comparison_df.to_csv(comparison_path, index=False)
    print(f"📄 Model comparison saved to: {comparison_path}")
    
    return {
        "model_a": model_a,
        "model_b": model_b,
        "prompt_name": prompt_name,
        "iteration": iteration,
        "pdf_union_count": len(pdf_union),
        "pdf_intersection_count": len(pdf_intersection),
        "webpage_union_count": len(webpage_union),
        "webpage_intersection_count": len(webpage_intersection),
        "total_union_count": total_union,
        "total_intersection_count": total_intersection
    }


In [8]:
def analyze_across_iterations(models, prompt_names, base_output_dir="results", iterations=[1, 2, 3]):
    """
    Analyzes results across all 3 iterations and creates final summary.
    """
    print(f"\n🔍 Analyzing results across {len(iterations)} iterations...")
    
    # 결과 저장할 리스트들
    iteration_trends = []
    model_comparisons = []
    
    # 각 iteration별로 데이터 수집
    for iteration in iterations:
        print(f"\n📊 Processing Iteration {iteration}...")
        
        for model in models:
            for prompt_name in prompt_names:
                # 메트릭 파일 로드
                metrics_path = os.path.join(base_output_dir, f"iteration_{iteration}", model, prompt_name, 
                                          "metrics", f"{model}_{prompt_name}_metrics.csv")
                
                if os.path.exists(metrics_path):
                    try:
                        metrics_df = pd.read_csv(metrics_path)
                        
                        # 메트릭을 딕셔너리로 변환
                        metrics_dict = dict(zip(metrics_df['Metric'], metrics_df['Value']))
                        
                        # iteration trends 데이터 추가
                        iteration_trends.append({
                            "Iteration": iteration,
                            "Model": model,
                            "Prompt": prompt_name,
                            "Total_Docs": metrics_dict.get("Total documents", 0),
                            "Total_PDFs": metrics_dict.get("Total PDFs", 0),
                            "Total_Webpages": metrics_dict.get("Total web page", 0),
                            "Overall_Recall": metrics_dict.get("Overall Recall", 0),
                            "Overall_Precision": metrics_dict.get("Overall Precision", 0),
                            "Avg_Recall_Per_Provider": metrics_dict.get("Average Recall per provider", 0),
                            "Avg_Precision_Per_Provider": metrics_dict.get("Average Precision per provider", 0)
                        })
                        
                        print(f"   ✅ Loaded metrics for {model}_{prompt_name}")
                        
                    except Exception as e:
                        print(f"   ❌ Failed to load metrics for {model}_{prompt_name}: {e}")
                else:
                    print(f"   ⚠️ Metrics file not found: {metrics_path}")
    
    # DataFrame 생성
    trends_df = pd.DataFrame(iteration_trends)
    
    # 모델별 최종 비교 (iteration 3 기준)
    final_iteration_data = trends_df[trends_df['Iteration'] == max(iterations)]
    
    # 모델별 평균 성능 계산
    model_summary = []
    for model in models:
        for prompt_name in prompt_names:
            model_data = final_iteration_data[
                (final_iteration_data['Model'] == model) & 
                (final_iteration_data['Prompt'] == prompt_name)
            ]
            
            if not model_data.empty:
                model_summary.append({
                    "Model": model,
                    "Prompt": prompt_name,
                    "Final_Total_Docs": model_data['Total_Docs'].iloc[0],  
                    "Final_Total_PDFs": model_data['Total_PDFs'].iloc[0],  
                    "Final_Total_Webpages": model_data['Total_Webpages'].iloc[0],
                    "Final_Overall_Recall": model_data['Overall_Recall'].iloc[0],
                    "Final_Overall_Precision": model_data['Overall_Precision'].iloc[0],
                    "Final_Avg_Recall_Per_Provider": model_data['Avg_Recall_Per_Provider'].iloc[0],
                    "Final_Avg_Precision_Per_Provider": model_data['Avg_Precision_Per_Provider'].iloc[0],
                    "Best_Iteration_Recall": trends_df[
                        (trends_df['Model'] == model) & 
                        (trends_df['Prompt'] == prompt_name)
                    ]['Overall_Recall'].max(),
                    "Best_Iteration_Precision": trends_df[
                        (trends_df['Model'] == model) & 
                        (trends_df['Prompt'] == prompt_name)
                    ]['Overall_Precision'].max()
                })
    
    model_comparison_df = pd.DataFrame(model_summary)
    
    # 전체 요약 통계
    overall_summary = []
    
    # 최고 성능 모델 찾기
    if not model_comparison_df.empty:
        best_recall_idx = model_comparison_df['Final_Overall_Recall'].idxmax()
        best_precision_idx = model_comparison_df['Final_Overall_Precision'].idxmax()
        
        best_recall_model = model_comparison_df.loc[best_recall_idx]
        best_precision_model = model_comparison_df.loc[best_precision_idx]
        
        overall_summary.extend([
            {
                "Metric": "Best Recall Model",
                "Value": f"{best_recall_model['Model']}_{best_recall_model['Prompt']}",
                "Score": best_recall_model['Final_Overall_Recall']
            },
            {
                "Metric": "Best Precision Model", 
                "Value": f"{best_precision_model['Model']}_{best_precision_model['Prompt']}",
                "Score": best_precision_model['Final_Overall_Precision']
            },
            {
                "Metric": "Average Final Recall",
                "Value": round(model_comparison_df['Final_Overall_Recall'].mean(), 4),
                "Score": ""
            },
            {
                "Metric": "Average Final Precision",
                "Value": round(model_comparison_df['Final_Overall_Precision'].mean(), 4),
                "Score": ""
            },
            {
                "Metric": "Total Model-Prompt Combinations",
                "Value": len(model_comparison_df),
                "Score": ""
            }
        ])
    
    overall_summary_df = pd.DataFrame(overall_summary)
    
    # 결과 저장
    final_analysis_folder = os.path.join(base_output_dir, "final_analysis")
    os.makedirs(final_analysis_folder, exist_ok=True)
    
    # 1. Iteration별 트렌드 저장
    trends_path = os.path.join(final_analysis_folder, "iteration_trends.csv")
    trends_df.to_csv(trends_path, index=False)
    print(f"📄 Iteration trends saved to: {trends_path}")
    
    # 2. 모델별 최종 비교 저장
    comparison_path = os.path.join(final_analysis_folder, "model_comparison.csv")
    model_comparison_df.to_csv(comparison_path, index=False)
    print(f"📄 Model comparison saved to: {comparison_path}")
    
    # 3. 전체 요약 저장
    summary_path = os.path.join(final_analysis_folder, "overall_summary.csv")
    overall_summary_df.to_csv(summary_path, index=False)
    print(f"📄 Overall summary saved to: {summary_path}")
    
    print(f"\n🎉 Final analysis complete! Results saved to: {final_analysis_folder}")
    return {
        "iteration_trends": trends_df,
        "model_comparison": model_comparison_df,
        "overall_summary": overall_summary_df,
        "final_analysis_folder": final_analysis_folder,
        "best_recall_model": f"{best_recall_model['Model']}_{best_recall_model['Prompt']}" if not model_comparison_df.empty else None,
        "best_precision_model": f"{best_precision_model['Model']}_{best_precision_model['Prompt']}" if not model_comparison_df.empty else None
    }


In [12]:
def run_single_iteration(iteration=1, providers=["Aetna"], experiment_id=1):
    """단일 iteration 실행"""
    base_output_dir = BASE_RESULT_DIR
    
    # 각 prompt와 모델에 대해 실행
    for prompt_name, prompt_fn in prompt_functions.items():
        for model in ["openai", "perplexity"]:
            print(f"\n=== [Iteration {iteration}] Running {model.upper()} for prompt '{prompt_name}' ===")
            
            # 새로운 함수 사용
            result = summarize_policy_retrieval_with_metrics(
                providers=providers,
                prompt_fn=prompt_fn,
                model=model,
                prompt_name=prompt_name,
                iteration=iteration,
                openai_client=gpt_client,
                perplexity_api_key=perplexity_api_key,
                manual_pdf_folder=MANUAL_FOLDER,
                known_links_csv=KNOWN_LINKS_CSV,  # 새로 추가 필요
                base_output_dir=base_output_dir
            )
            
            print(f"✅ {model}_{prompt_name} completed for iteration {iteration}")
        
        # 각 prompt에 대해 모델 간 비교
        compare_models_union_intersection(
            model_a="openai",
            model_b="perplexity", 
            prompt_name=prompt_name,
            iteration=iteration,
            base_output_dir=base_output_dir
        )


In [13]:
def run_policy_experiments_multiple_iterations(iterations=[1, 2, 3], providers=["United Healthcare"]):
    """3번 iteration 실행 및 최종 분석"""
    base_output_dir = BASE_RESULT_DIR
    
    # 각 iteration 실행
    for iteration in iterations:
        print(f"\n🚀 Running policy retrieval iteration {iteration}...\n")
        run_single_iteration(iteration=iteration, providers=providers)
    
    # 최종 통합 분석
    print(f"\n🔍 Running final analysis across all iterations...")
    final_result = analyze_across_iterations(
        models=["openai", "perplexity"],
        prompt_names=list(prompt_functions.keys()),
        base_output_dir=base_output_dir,
        iterations=iterations
    )
    
    print(f"🎉 All experiments completed!")
    return final_result



In [14]:
run_single_iteration(iteration=1, providers=["Aetna"])


=== [Iteration 1] Running OPENAI for prompt 'baseline' ===

📊 [Iteration 1] Starting retrieval summary for 1 providers...

🔍 [Iteration 1] Searching for: Aetna
🔁 Attempt 1 (openai)...

🧾 OPENAI raw response for 'Aetna':
```json
{
  "pdf_links": [],
  "webpage_links": [
    "https://www.aetna.com/cpb/medical/data/100_199/0140.html",
    "https://www.aetna.com/cpb/medical/data/100_199/0189.html",
    "https://www.aetna.com/cpb/medical/data/700_799/0715.html"
  ]
}
```

📂 Loading known PDFs from: /home/cptaswadu/RESCUE-n8n/insurance/insurance_policy
   Found 12 PDFs for United Healthcare
   Found 43 PDFs for BCBS FEP
   Found 70 PDFs for Cigna
📂 Loading known webpage links from: /home/cptaswadu/RESCUE-n8n/insurance/provider_policy.csv
⚠️  No known PDFs for Aetna
📋 Known data for Aetna:
   Known PDFs: 0
   Known webpages: 12
✅ Webpage Match: https://www.aetna.com/cpb/medical/data/100_199/0140.html
❌ Webpage New/Unmatched: https://www.aetna.com/cpb/medical/data/100_199/0189.html
✅ Webpage 

In [15]:
# 보험사 리스트 정의
providers = ["United Healthcare", "Aetna", "Cigna", "BCBS Federal Employee Program"]

# 3번의 iteration으로 실험 실행
final_result = run_policy_experiments_multiple_iterations(
    iterations=[1, 2, 3], 
    providers=providers
)

# 결과 출력
print("\n📊 Final Experiment Results:")
print("=" * 50)
for key, value in final_result.items():
    print(f"{key}: {value}")



🚀 Running policy retrieval iteration 1...


=== [Iteration 1] Running OPENAI for prompt 'baseline' ===

📊 [Iteration 1] Starting retrieval summary for 4 providers...

🔍 [Iteration 1] Searching for: United Healthcare
🔁 Attempt 1 (openai)...

🧾 OPENAI raw response for 'United Healthcare':
```json
{
  "pdf_links": [
    "https://www.uhcprovider.com/content/dam/provider/docs/public/policies/comm-medical-drug/genetic-testing-for-hereditary-cancer.pdf",
    "https://www.uhcprovider.com/content/dam/provider/docs/public/policies/comm-medical-drug/genetic-testing-for-hereditary-cancer.pdf"
  ],
  "webpage_links": [
    "https://www.uhcprovider.com/en/prior-auth-advance-notification/genetic-molecular-lab.html",
    "https://www.uhcprovider.com/en/resource-library/news/2025/genetic-testing-codes-req-prior-auth.html",
    "https://www.uhcprovider.com/en/resource-library/news/2024/changes-genetic-molecular-testing-coverage-pa-requirements.html"
  ]
}
```

📂 Loading known PDFs from: /home/cptaswadu

In [9]:
def create_final_summary_table(models, prompt_names, base_output_dir="/home/cptaswadu/RESCUE-n8n/insurance/results/policy_retrieval_fin", iterations=[1, 2, 3]):
    print(f"\n📊 Creating final summary table across {len(iterations)} iterations...")
    
    # 각 모델별 데이터 수집
    model_data = {}
    
    for model in models:
        for prompt_name in prompt_names:
            model_key = f"{model}_{prompt_name}"
            
            # 모든 iteration의 메트릭 수집
            all_metrics = []
            
            for iteration in iterations:
                detailed_path = os.path.join(base_output_dir, f"iteration_{iteration}", model, prompt_name, "raw_results", f"{model}_{prompt_name}_detailed.csv")
                
                if os.path.exists(detailed_path):
                    try:
                        df = pd.read_csv(detailed_path)
                        
                        # 메트릭 직접 계산
                        total_links = df["Total Links"].sum()
                        pdf_count = df["PDF Count"].sum()
                        webpage_count = df["Webpage Count"].sum()
                        total_matches = df["Total Matches"].sum()
                        total_known = df["Total Known"].sum()
                        
                        # 평균 계산
                        avg_total_docs = df["Total Links"].mean()
                        avg_pdfs = df["PDF Count"].mean()
                        avg_webpages = df["Webpage Count"].mean()
                        
                        # Recall/Precision 계산
                        overall_recall = total_matches / total_known if total_known > 0 else 0
                        overall_precision = total_matches / total_links if total_links > 0 else 0
                        
                        # Provider별 recall/precision 계산
                        provider_recalls = []
                        provider_precisions = []
                        
                        for _, row in df.iterrows():
                            found = row["Total Links"]
                            matched = row["Total Matches"]
                            known = row["Total Known"]
                            
                            recall = matched / known if known > 0 else 0
                            precision = matched / found if found > 0 else 0
                            
                            provider_recalls.append(recall)
                            provider_precisions.append(precision)
                        
                        avg_recall_per_provider = np.mean(provider_recalls) if provider_recalls else 0
                        avg_precision_per_provider = np.mean(provider_precisions) if provider_precisions else 0
                        
                        # 결과 저장
                        all_metrics.append({
                            "Total documents": total_links,
                            "Total PDFs": pdf_count,
                            "Total web page": webpage_count,
                            "Total documents (average #)": avg_total_docs,
                            "Total PDFs (average #)": avg_pdfs,
                            "Total web page (average #)": avg_webpages,
                            "Overall Recall": overall_recall,
                            "Overall Precision": overall_precision,
                            "Average Recall per provider": avg_recall_per_provider,
                            "Average Precision per provider": avg_precision_per_provider
                        })
                        
                        print(f"   ✅ Loaded {model_key} iteration {iteration}")
                        
                    except Exception as e:
                        print(f"   ❌ Failed to load {model_key} iteration {iteration}: {e}")
                else:
                    print(f"   ⚠️ File not found: {detailed_path}")
            
            # 각 모델별 평균과 표준편차 계산
            if all_metrics:
                docs_values = [m.get("Total documents", 0) for m in all_metrics]
                pdfs_values = [m.get("Total PDFs", 0) for m in all_metrics]
                webpages_values = [m.get("Total web page", 0) for m in all_metrics]
                recall_values = [m.get("Overall Recall", 0) for m in all_metrics]
                precision_values = [m.get("Overall Precision", 0) for m in all_metrics]
                avg_recall_values = [m.get("Average Recall per provider", 0) for m in all_metrics]
                avg_precision_values = [m.get("Average Precision per provider", 0) for m in all_metrics]
                
                model_data[model_key] = {
                    "total_docs_mean": np.mean(docs_values),
                    "total_pdfs_mean": np.mean(pdfs_values),
                    "total_webpages_mean": np.mean(webpages_values),
                    "overall_recall_mean": np.mean(recall_values),
                    "overall_recall_std": np.std(recall_values),
                    "overall_precision_mean": np.mean(precision_values),
                    "overall_precision_std": np.std(precision_values),
                    "avg_recall_per_provider": np.mean(avg_recall_values),
                    "avg_precision_per_provider": np.mean(avg_precision_values)
                }
                print(f"   📊 {model_key} metrics calculated")
    
    # Union/Intersection 계산 (모든 iteration과 prompt 조합에 대해)
    union_docs_values = []
    union_pdfs_values = []
    union_webpages_values = []
    intersection_docs_values = []
    intersection_pdfs_values = []
    intersection_webpages_values = []
    
    for iteration in iterations:
        for prompt_name in prompt_names:
            # 각 iteration과 prompt에 대해 Union/Intersection 계산
            comparison_path = os.path.join(base_output_dir, f"iteration_{iteration}", models[0], prompt_name, "comparisons")
            union_docs = union_pdfs = union_webpages = 0
            intersection_docs = intersection_pdfs = intersection_webpages = 0

            if os.path.exists(comparison_path):
                try:
                    comparison_files = [f for f in os.listdir(comparison_path) if f.endswith('.csv')]
                    if comparison_files:
                        comp_df = pd.read_csv(os.path.join(comparison_path, comparison_files[0]))
                        if not comp_df.empty:
                            union_docs = comp_df['Total Union Count'].iloc[0]
                            union_pdfs = comp_df['PDF Union Count'].iloc[0]
                            union_webpages = comp_df['Webpage Union Count'].iloc[0]
                            intersection_docs = comp_df['Total Intersection Count'].iloc[0]
                            intersection_pdfs = comp_df['PDF Intersection Count'].iloc[0]
                            intersection_webpages = comp_df['Webpage Intersection Count'].iloc[0]
                except Exception as e:
                    print(f"   ⚠️ Could not load comparison for iteration {iteration}, prompt {prompt_name}: {e}")
            
            union_docs_values.append(union_docs)
            union_pdfs_values.append(union_pdfs)
            union_webpages_values.append(union_webpages)
            intersection_docs_values.append(intersection_docs)
            intersection_pdfs_values.append(intersection_pdfs)
            intersection_webpages_values.append(intersection_webpages)
    
    # Union/Intersection 평균 계산
    union_intersection_data = {
        "Union": {
            "total_docs_mean": np.mean(union_docs_values) if union_docs_values else 0,
            "total_pdfs_mean": np.mean(union_pdfs_values) if union_pdfs_values else 0,
            "total_webpages_mean": np.mean(union_webpages_values) if union_webpages_values else 0,
            "overall_recall_mean": "-",
                        "overall_recall_std": "-",
            "overall_precision_mean": "-",
            "overall_precision_std": "-",
            "avg_recall_per_provider": "-",
            "avg_precision_per_provider": "-"
        },
        "Intersection": {
            "total_docs_mean": np.mean(intersection_docs_values) if intersection_docs_values else 0,
            "total_pdfs_mean": np.mean(intersection_pdfs_values) if intersection_pdfs_values else 0,
            "total_webpages_mean": np.mean(intersection_webpages_values) if intersection_webpages_values else 0,
            "overall_recall_mean": "-",
            "overall_recall_std": "-",
            "overall_precision_mean": "-",
            "overall_precision_std": "-",
            "avg_recall_per_provider": "-",
            "avg_precision_per_provider": "-"
        }
    }
    
    # 최종 테이블 생성
    columns = []
    for model in models:
        for prompt_name in prompt_names:
            columns.append(f"{model}_{prompt_name}")
    columns.extend(["Union", "Intersection"])
    
    # 데이터 정리
    final_data = {}
    for col in columns:
        if col in model_data:
            final_data[col] = model_data[col]
        elif col in union_intersection_data:
            final_data[col] = union_intersection_data[col]
    
    # 테이블 생성
    rows = [
        "Total documents",
        "Total PDF",
        "Total web page",
        "Overall mean Recall (std)",
        "Overall mean Precision (std)",
        "Average Recall per provider",
        "Average Precision per provider"
    ]
    
    # 최종 결과 DataFrame 생성
    result_data = []
    
    for row in rows:
        row_data = {"Metric": row}
        
        for col in columns:
            if col in final_data:
                data = final_data[col]
                
                if row == "Total documents":
                    row_data[col] = f"{data['total_docs_mean']:.1f}"
                elif row == "Total PDF":
                    row_data[col] = f"{data['total_pdfs_mean']:.1f}"
                elif row == "Total web page":
                    row_data[col] = f"{data['total_webpages_mean']:.1f}"
                elif row == "Overall mean Recall (std)":
                    if data['overall_recall_mean'] == "-":
                        row_data[col] = "-"
                    else:
                        row_data[col] = f"{data['overall_recall_mean']:.1%} ({data['overall_recall_std']:.1%})"
                elif row == "Overall mean Precision (std)":
                    if data['overall_precision_mean'] == "-":
                        row_data[col] = "-"
                    else:
                        row_data[col] = f"{data['overall_precision_mean']:.1%} ({data['overall_precision_std']:.1%})"
                elif row == "Average Recall per provider":
                    if data['avg_recall_per_provider'] == "-":
                        row_data[col] = "-"
                    else:
                        row_data[col] = f"{data['avg_recall_per_provider']:.1%}"
                elif row == "Average Precision per provider":
                    if data['avg_precision_per_provider'] == "-":
                        row_data[col] = "-"
                    else:
                        row_data[col] = f"{data['avg_precision_per_provider']:.1%}"
            else:
                row_data[col] = "-"
        
        result_data.append(row_data)
    
    # DataFrame 생성
    final_table = pd.DataFrame(result_data)
    
    # 결과 저장
    final_analysis_folder = os.path.join(base_output_dir, "final_analysis")
    os.makedirs(final_analysis_folder, exist_ok=True)
    
    summary_table_path = os.path.join(final_analysis_folder, "final_summary_table.csv")
    final_table.to_csv(summary_table_path, index=False)
    
    print(f"📄 Final summary table saved to: {summary_table_path}")
    print(f"\n📊 Final Summary Table:")
    print(final_table.to_string(index=False))
    
    return final_table


In [10]:
models = ["openai", "perplexity"]
prompt_names = ["baseline", "keyword", "verified"]
iterations = [1, 2, 3]

final_table = create_final_summary_table(models, prompt_names, iterations=iterations)



📊 Creating final summary table across 3 iterations...
   ✅ Loaded openai_baseline iteration 1
   ✅ Loaded openai_baseline iteration 2
   ✅ Loaded openai_baseline iteration 3
   📊 openai_baseline metrics calculated
   ✅ Loaded openai_keyword iteration 1
   ✅ Loaded openai_keyword iteration 2
   ✅ Loaded openai_keyword iteration 3
   📊 openai_keyword metrics calculated
   ✅ Loaded openai_verified iteration 1
   ✅ Loaded openai_verified iteration 2
   ✅ Loaded openai_verified iteration 3
   📊 openai_verified metrics calculated
   ✅ Loaded perplexity_baseline iteration 1
   ✅ Loaded perplexity_baseline iteration 2
   ✅ Loaded perplexity_baseline iteration 3
   📊 perplexity_baseline metrics calculated
   ✅ Loaded perplexity_keyword iteration 1
   ✅ Loaded perplexity_keyword iteration 2
   ✅ Loaded perplexity_keyword iteration 3
   📊 perplexity_keyword metrics calculated
   ✅ Loaded perplexity_verified iteration 1
   ✅ Loaded perplexity_verified iteration 2
   ✅ Loaded perplexity_verified i

In [11]:
def create_model_summary_table(models, prompt_names, base_output_dir="/home/cptaswadu/RESCUE-n8n/insurance/results/policy_retrieval_fin", iterations=[1, 2, 3]):
    print(f"\n📊 Creating model summary table across {len(iterations)} iterations...")
    
    # 각 모델별 데이터 수집 (모든 prompt 통합)
    model_data = {}
    
    for model in models:
        # 해당 모델의 모든 prompt와 iteration 데이터 수집
        all_docs = []
        all_pdfs = []
        all_webpages = []
        all_recalls = []
        all_precisions = []
        all_provider_recalls = []
        all_provider_precisions = []
        
        for prompt_name in prompt_names:
            for iteration in iterations:
                detailed_path = os.path.join(base_output_dir, f"iteration_{iteration}", model, prompt_name, "raw_results", f"{model}_{prompt_name}_detailed.csv")
                
                if os.path.exists(detailed_path):
                    try:
                        df = pd.read_csv(detailed_path)
                        
                        # 메트릭 계산
                        total_links = df["Total Links"].sum()
                        pdf_count = df["PDF Count"].sum()
                        webpage_count = df["Webpage Count"].sum()
                        total_matches = df["Total Matches"].sum()
                        total_known = df["Total Known"].sum()
                        
                        # 평균 계산
                        avg_total_docs = df["Total Links"].mean()
                        avg_pdfs = df["PDF Count"].mean()
                        avg_webpages = df["Webpage Count"].mean()
                        
                        # Recall/Precision 계산
                        overall_recall = total_matches / total_known if total_known > 0 else 0
                        overall_precision = total_matches / total_links if total_links > 0 else 0
                        
                        # Provider별 recall/precision 계산
                        provider_recalls = []
                        provider_precisions = []
                        
                        for _, row in df.iterrows():
                            found = row["Total Links"]
                            matched = row["Total Matches"]
                            known = row["Total Known"]
                            
                            recall = matched / known if known > 0 else 0
                            precision = matched / found if found > 0 else 0
                            
                            provider_recalls.append(recall)
                            provider_precisions.append(precision)
                        
                        avg_recall_per_provider = np.mean(provider_recalls) if provider_recalls else 0
                        avg_precision_per_provider = np.mean(provider_precisions) if provider_precisions else 0
                        
                        # 데이터 추가
                        all_docs.append(total_links)
                        all_pdfs.append(pdf_count)
                        all_webpages.append(webpage_count)
                        all_recalls.append(overall_recall)
                        all_precisions.append(overall_precision)
                        all_provider_recalls.append(avg_recall_per_provider)
                        all_provider_precisions.append(avg_precision_per_provider)
                        
                        print(f"   ✅ Loaded {model} {prompt_name} iteration {iteration}")
                        
                    except Exception as e:
                        print(f"   ❌ Failed to load {model} {prompt_name} iteration {iteration}: {e}")
                else:
                    print(f"   ⚠️ File not found: {detailed_path}")
        
        # 모델별 전체 평균과 표준편차 계산
        if all_docs:
            model_data[model] = {
                "total_docs_mean": np.mean(all_docs),
                "total_pdfs_mean": np.mean(all_pdfs),
                "total_webpages_mean": np.mean(all_webpages),
                "overall_recall_mean": np.mean(all_recalls),
                "overall_recall_std": np.std(all_recalls),
                "overall_precision_mean": np.mean(all_precisions),
                "overall_precision_std": np.std(all_precisions),
                "avg_recall_per_provider": np.mean(all_provider_recalls),
                "avg_precision_per_provider": np.mean(all_provider_precisions)
            }
            print(f"   📊 {model} metrics calculated")
    

        union_docs_values = []
        union_pdfs_values = []
        union_webpages_values = []
        intersection_docs_values = []
        intersection_pdfs_values = []
        intersection_webpages_values = []   

        for iteration in iterations:
            for prompt_name in prompt_names:
                comparison_path = os.path.join(base_output_dir, f"iteration_{iteration}", models[0], prompt_name, "comparisons")
        
                if os.path.exists(comparison_path):
                    try:
                        comparison_files = [f for f in os.listdir(comparison_path) if f.endswith('.csv')]
                        if comparison_files:
                            comp_df = pd.read_csv(os.path.join(comparison_path, comparison_files[0]))
                            if not comp_df.empty:
                                union_docs_values.append(comp_df['Total Union Count'].iloc[0])
                                union_pdfs_values.append(comp_df['PDF Union Count'].iloc[0])
                                union_webpages_values.append(comp_df['Webpage Union Count'].iloc[0])
                                intersection_docs_values.append(comp_df['Total Intersection Count'].iloc[0])
                                intersection_pdfs_values.append(comp_df['PDF Intersection Count'].iloc[0])
                                intersection_webpages_values.append(comp_df['Webpage Intersection Count'].iloc[0])
                    except Exception as e:
                        print(f"   ⚠️ Could not load comparison: {e}")

# Union/Intersection 데이터
    union_intersection_data = {
        "Union": {
            "total_docs_mean": np.mean(union_docs_values) if union_docs_values else 0,
            "total_pdfs_mean": np.mean(union_pdfs_values) if union_pdfs_values else 0,
            "total_webpages_mean": np.mean(union_webpages_values) if union_webpages_values else 0,
            "overall_recall_mean": "-",
            "overall_recall_std": "-",
            "overall_precision_mean": "-",
            "overall_precision_std": "-",
            "avg_recall_per_provider": "-",
            "avg_precision_per_provider": "-"
        },
        "Intersection": {
            "total_docs_mean": np.mean(intersection_docs_values) if intersection_docs_values else 0,
            "total_pdfs_mean": np.mean(intersection_pdfs_values) if intersection_pdfs_values else 0,
            "total_webpages_mean": np.mean(intersection_webpages_values) if intersection_webpages_values else 0,
            "overall_recall_mean": "-",
            "overall_recall_std": "-",
            "overall_precision_mean": "-",
            "overall_precision_std": "-",
            "avg_recall_per_provider": "-",
            "avg_precision_per_provider": "-"
        }
    }
        
    # 최종 테이블 생성
    columns = models + ["Union", "Intersection"]
    
    # 데이터 정리
    final_data = {}
    for col in columns:
        if col in model_data:
            final_data[col] = model_data[col]
        elif col in union_intersection_data:
            final_data[col] = union_intersection_data[col]
    
    # 테이블 생성
    rows = [
        "Total documents",
        "Total PDF", 
        "Total web page",
        "Overall mean Recall (std)",
        "Overall mean Precision (std)",
        "Average Recall per provider",
        "Average Precision per provider"
    ]
    
    # 최종 결과 DataFrame 생성
    result_data = []
    
    for row in rows:
        row_data = {"Metric": row}
        
        for col in columns:
            if col in final_data:
                data = final_data[col]
                
                if row == "Total documents":
                    row_data[col] = f"{data['total_docs_mean']:.1f}" if data['total_docs_mean'] != "-" else "-"
                elif row == "Total PDF":
                    row_data[col] = f"{data['total_pdfs_mean']:.1f}" if data['total_pdfs_mean'] != "-" else "-"
                elif row == "Total web page":
                    row_data[col] = f"{data['total_webpages_mean']:.1f}" if data['total_webpages_mean'] != "-" else "-"
                elif row == "Overall mean Recall (std)":
                    if data['overall_recall_mean'] == "-":
                        row_data[col] = "-"
                    else:
                        row_data[col] = f"{data['overall_recall_mean']:.1%} ({data['overall_recall_std']:.1%})"
                elif row == "Overall mean Precision (std)":
                    if data['overall_precision_mean'] == "-":
                        row_data[col] = "-"
                    else:
                        row_data[col] = f"{data['overall_precision_mean']:.1%} ({data['overall_precision_std']:.1%})"
                elif row == "Average Recall per provider":
                    if data['avg_recall_per_provider'] == "-":
                        row_data[col] = "-"
                    else:
                        row_data[col] = f"{data['avg_recall_per_provider']:.1%}"
                elif row == "Average Precision per provider":
                    if data['avg_precision_per_provider'] == "-":
                        row_data[col] = "-"
                    else:
                        row_data[col] = f"{data['avg_precision_per_provider']:.1%}"
            else:
                row_data[col] = "-"
        
        result_data.append(row_data)
    
    # DataFrame 생성
    final_table = pd.DataFrame(result_data)
    
    # 결과 저장
    final_analysis_folder = os.path.join(base_output_dir, "final_analysis")
    os.makedirs(final_analysis_folder, exist_ok=True)
    
    summary_table_path = os.path.join(final_analysis_folder, "model_summary_table.csv")
    final_table.to_csv(summary_table_path, index=False)
    
    print(f"📄 Model summary table saved to: {summary_table_path}")
    print(f"\n📊 Model Summary Table:")
    print(final_table.to_string(index=False))
    
    return final_table


In [12]:
# 모델별 요약 테이블 생성
models = ["openai", "perplexity"]
prompt_names = ["baseline", "keyword", "verified"]
iterations = [1, 2, 3]

model_summary_table = create_model_summary_table(models, prompt_names, iterations=iterations)



📊 Creating model summary table across 3 iterations...
   ✅ Loaded openai baseline iteration 1
   ✅ Loaded openai baseline iteration 2
   ✅ Loaded openai baseline iteration 3
   ✅ Loaded openai keyword iteration 1
   ✅ Loaded openai keyword iteration 2
   ✅ Loaded openai keyword iteration 3
   ✅ Loaded openai verified iteration 1
   ✅ Loaded openai verified iteration 2
   ✅ Loaded openai verified iteration 3
   📊 openai metrics calculated
   ✅ Loaded perplexity baseline iteration 1
   ✅ Loaded perplexity baseline iteration 2
   ✅ Loaded perplexity baseline iteration 3
   ✅ Loaded perplexity keyword iteration 1
   ✅ Loaded perplexity keyword iteration 2
   ✅ Loaded perplexity keyword iteration 3
   ✅ Loaded perplexity verified iteration 1
   ✅ Loaded perplexity verified iteration 2
   ✅ Loaded perplexity verified iteration 3
   📊 perplexity metrics calculated
📄 Model summary table saved to: /home/cptaswadu/RESCUE-n8n/insurance/results/policy_retrieval_fin/final_analysis/model_summary_ta

In [2]:
def calculate_union_intersection(models, prompt_names, base_output_dir, iterations=[1, 2, 3]):
    """
    두 모델의 전체 검색 결과에서 Union과 Intersection을 계산하는 함수
    """
    print(f"\n🔄 Calculating Union/Intersection between models...")
    
    # 각 모델별로 전체 검색 결과 수집
    model_all_results = {}
    
    for model in models:
        all_documents = set()  # 중복 제거를 위한 set
        all_pdfs = set()
        all_webpages = set()
        
        for iteration in iterations:
            for prompt_name in prompt_names:
                detailed_path = os.path.join(base_output_dir, f"iteration_{iteration}", model, prompt_name, "raw_results", f"{model}_{prompt_name}_detailed.csv")
                
                if os.path.exists(detailed_path):
                    try:
                        df = pd.read_csv(detailed_path)
                        
                        # 각 행의 검색 결과에서 링크 정보 추출
                        for _, row in df.iterrows():
                            # PDF Links 추출
                            pdf_links_str = row.get("PDF Links", "[]")
                            if pdf_links_str and pdf_links_str != "[]":
                                try:
                                    pdf_links = ast.literal_eval(pdf_links_str)
                                    for link in pdf_links:
                                        all_pdfs.add(link)
                                        all_documents.add(link)
                                except:
                                    pass
                            
                            # Webpage Links 추출
                            webpage_links_str = row.get("Webpage Links", "[]")
                            if webpage_links_str and webpage_links_str != "[]":
                                try:
                                    webpage_links = ast.literal_eval(webpage_links_str)
                                    for link in webpage_links:
                                        all_webpages.add(link)
                                        all_documents.add(link)
                                except:
                                    pass
                                    
                    except Exception as e:
                        print(f"   ⚠️ Could not load {model} iteration {iteration}, prompt {prompt_name}: {e}")
        
        model_all_results[model] = {
            'all_documents': all_documents,
            'all_pdfs': all_pdfs,
            'all_webpages': all_webpages
        }
        
        print(f"   ✅ {model}: {len(all_documents)} total documents ({len(all_pdfs)} PDFs, {len(all_webpages)} webpages)")
    
    # 두 모델 간 Union/Intersection 계산
    if len(models) == 2:
        model1, model2 = models[0], models[1]
        
        # 전체 문서 Union/Intersection
        union_docs = model_all_results[model1]['all_documents'] | model_all_results[model2]['all_documents']
        intersection_docs = model_all_results[model1]['all_documents'] & model_all_results[model2]['all_documents']
        
        # PDF Union/Intersection
        union_pdfs = model_all_results[model1]['all_pdfs'] | model_all_results[model2]['all_pdfs']
        intersection_pdfs = model_all_results[model1]['all_pdfs'] & model_all_results[model2]['all_pdfs']
        
        # Webpage Union/Intersection
        union_webpages = model_all_results[model1]['all_webpages'] | model_all_results[model2]['all_webpages']
        intersection_webpages = model_all_results[model1]['all_webpages'] & model_all_results[model2]['all_webpages']
        
        # 결과 정리
        union_intersection_data = {
            "Union": {
                "total_docs_mean": len(union_docs),
                "total_pdfs_mean": len(union_pdfs),
                "total_webpages_mean": len(union_webpages),
                "overall_recall_mean": "-",
                "overall_recall_std": "-",
                "overall_precision_mean": "-",
                "overall_precision_std": "-",
                "avg_recall_per_provider": "-",
                "avg_precision_per_provider": "-"
            },
            "Intersection": {
                "total_docs_mean": len(intersection_docs),
                "total_pdfs_mean": len(intersection_pdfs),
                "total_webpages_mean": len(intersection_webpages),
                "overall_recall_mean": "-",
                "overall_recall_std": "-",
                "overall_precision_mean": "-",
                "overall_precision_std": "-",
                "avg_recall_per_provider": "-",
                "avg_precision_per_provider": "-"
            }
        }
        
        # 결과 출력
        print(f"\n📊 Union/Intersection Results:")
        print(f"   Union: {len(union_docs)} documents ({len(union_pdfs)} PDFs, {len(union_webpages)} webpages)")
        print(f"   Intersection: {len(intersection_docs)} documents ({len(intersection_pdfs)} PDFs, {len(intersection_webpages)} webpages)")
        
        # 링크 데이터 저장
        final_analysis_folder = os.path.join(base_output_dir, "final_analysis")
        os.makedirs(final_analysis_folder, exist_ok=True)
        
        # 각 모델별 링크 저장
        for model in models:
            model_links_data = {
                'all_documents': list(model_all_results[model]['all_documents']),
                'all_pdfs': list(model_all_results[model]['all_pdfs']),
                'all_webpages': list(model_all_results[model]['all_webpages'])
            }
            
            # JSON 파일로 저장
            import json
            model_links_path = os.path.join(final_analysis_folder, f"{model}_all_links.json")
            with open(model_links_path, 'w', encoding='utf-8') as f:
                json.dump(model_links_data, f, indent=2, ensure_ascii=False)
            print(f"   📄 {model} links saved to: {model_links_path}")
        
        # Union/Intersection 링크 저장
        union_intersection_links = {
            'union_all_documents': list(union_docs),
            'union_pdfs': list(union_pdfs),
            'union_webpages': list(union_webpages),
            'intersection_all_documents': list(intersection_docs),
            'intersection_pdfs': list(intersection_pdfs),
            'intersection_webpages': list(intersection_webpages)
        }
        
        union_intersection_path = os.path.join(final_analysis_folder, "union_intersection_links.json")
        with open(union_intersection_path, 'w', encoding='utf-8') as f:
            json.dump(union_intersection_links, f, indent=2, ensure_ascii=False)
        print(f"   📄 Union/Intersection links saved to: {union_intersection_path}")
        
        return union_intersection_data
    
    else:
        print(f"   ⚠️ Need exactly 2 models for Union/Intersection calculation, got {len(models)}")
        return {
            "Union": {
                "total_docs_mean": 0,
                "total_pdfs_mean": 0,
                "total_webpages_mean": 0,
                "overall_precision_mean": "-",
                "overall_precision_std": "-",
                "avg_recall_per_provider": "-",
                "avg_precision_per_provider": "-"
            },
            "Intersection": {
                "total_docs_mean": 0,
                "total_pdfs_mean": 0,
                "total_webpages_mean": 0,
                "overall_recall_mean": "-",
                "overall_recall_std": "-",
                "overall_precision_mean": "-",
                "overall_precision_std": "-",
                "avg_recall_per_provider": "-",
                "avg_precision_per_provider": "-"
            }
        }



In [3]:
models = ["openai", "perplexity"]
prompt_names = ["baseline", "keyword", "verified"]
base_output_dir = "/home/cptaswadu/RESCUE-n8n/insurance/results/policy_retrieval_fin"
iterations = [1, 2, 3]
result = calculate_union_intersection(models, prompt_names, base_output_dir, iterations)


🔄 Calculating Union/Intersection between models...
   ✅ openai: 73 total documents (37 PDFs, 38 webpages)
   ✅ perplexity: 18 total documents (10 PDFs, 8 webpages)

📊 Union/Intersection Results:
   Union: 86 documents (47 PDFs, 41 webpages)
   Intersection: 5 documents (0 PDFs, 5 webpages)
   📄 openai links saved to: /home/cptaswadu/RESCUE-n8n/insurance/results/policy_retrieval_fin/final_analysis/openai_all_links.json
   📄 perplexity links saved to: /home/cptaswadu/RESCUE-n8n/insurance/results/policy_retrieval_fin/final_analysis/perplexity_all_links.json
   📄 Union/Intersection links saved to: /home/cptaswadu/RESCUE-n8n/insurance/results/policy_retrieval_fin/final_analysis/union_intersection_links.json
